In [1]:
import sys
print('google.colab' in sys.modules)
if 'google.colab' in sys.modules:
    !git clone https://github.com/theoliver7/uzh-aml.git
    %cd uzh-aml
    %load_ext tensorboard
    !pip install -r colab-requirements.txt
    


False


In [2]:
import glob
import importlib
import os
import time

import numpy as np
import torch
import torch.nn.functional as F
from sklearn.model_selection import KFold
from torch.utils.data import random_split
from torch_geometric.datasets import TUDataset
from torch_geometric.loader import DataLoader

# IMPORT THE MODEL TO BE TRAINED !!!
import models.hgp_sl_param.models as models

In [3]:
# Refresh model so that code is updated directly
importlib.reload(models)

<module 'models.hgp_sl_param.models' from 'C:\\Users\\carlo\\Desktop\\Programmieren\\aml\\uzh-aml\\models\\hgp_sl_param\\models.py'>

In [4]:
args = {
    'device': "cuda",
    'seed': 777,
    'batch_size': 64,
    'lr': 0.0001,
    'weight_decay': 0.001,
    'epochs': 1000,
    'patience': 100,
    'nhid': 128,
    'pooling_ratio': 0.3,
    'dropout_ratio': 0.5,
    'sample_neighbor': True,
    'sparse_attention': True,
    'structure_learning': True,
    'negative_slope': 0.2,
    'lamb': 1.0,
    'dist': "man",
    'num_layers': 2,
    'layers_readout': [0, 1] # Assume we have 3 layers (so K = 3) and you want all 3 hidden representations to be part of the readout, you need to set this to [0, 1, 2]. If you only want the first and last layer, you need to set this to [0, 2]
}


In [5]:
torch.set_printoptions(edgeitems=10)
print(torch.cuda.is_available())
if torch.cuda.is_available():
    args["device"] = 'cuda'
# SET THE RANDOM SEED
torch.manual_seed(args['seed'])

dataset = TUDataset(os.path.join('../../data', "DD"), name="DD", use_node_attr=True)
args["num_classes"] = dataset.num_classes
args["num_features"] = dataset.num_features
num_test = int(len(dataset) * 0.1)
num_train = len(dataset) - num_test
train_dataset, _ = torch.utils.data.random_split(dataset, [num_train, num_test])
kfold = KFold(n_splits=5, shuffle=True, random_state=args['seed'])

True


In [6]:
def compute_test(model, loader):
    model.eval()
    correct = 0.0
    loss_test = 0.0
    for data in loader:
        data = data.to(args["device"])
        out = model(data)
        pred = out.max(dim=1)[1]
        correct += pred.eq(data.y).sum().item()
        loss_test += F.nll_loss(out, data.y).item()
    return correct / len(loader.dataset), loss_test

In [7]:
def train_fold(fold, train_idx, val_idx):
    train_subset = torch.utils.data.Subset(train_dataset, train_idx)
    val_subset = torch.utils.data.Subset(train_dataset, val_idx)
    
    train_loader = DataLoader(train_subset, batch_size=args["batch_size"], shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=args["batch_size"], shuffle=False)
    print(args)
    model = models.Model(args).to(args["device"])
 
    optimizer = torch.optim.Adam(model.parameters(), lr=args["lr"], weight_decay=args["weight_decay"])
    min_loss = 1e10
    patience_cnt = 0
    val_loss_values = []
    val_acc_values = []
    best_epoch = 0

    t = time.time()
    model.train()
    for epoch in range(args["epochs"]):
        loss_train = 0.0
        correct = 0
        for i, data in enumerate(train_loader):
            optimizer.zero_grad()
            data = data.to(args["device"])
            out = model(data)
            loss = F.nll_loss(out, data.y)
            loss.backward()
            optimizer.step()
            loss_train += loss.item()
            pred = out.max(dim=1)[1]
            correct += pred.eq(data.y).sum().item()
        acc_train = correct / len(train_loader.dataset)
        acc_val, loss_val = compute_test(model, val_loader)
        print('Fold:{:04d}'.format(fold), 'Epoch: {:04d}'.format(epoch), 'loss_train: {:.6f}'.format(loss_train),
              'acc_train: {:.6f}'.format(acc_train), 'loss_val: {:.6f}'.format(loss_val),
              'acc_val: {:.6f}'.format(acc_val), 'time: {:.6f}s'.format(time.time() - t))
        
        val_loss_values.append(loss_val)
        val_acc_values.append(acc_val)
        if val_loss_values[-1] < min_loss:
            min_loss = val_loss_values[-1]
            best_epoch = epoch
            patience_cnt = 0
        else:
            patience_cnt += 1

        if patience_cnt == args["patience"]:
            break

        
    print('Optimization Finished! Total time elapsed: {:.6f}'.format(time.time() - t))
    print('Fold:{:04d}'.format(fold), 'Best Epoch: {:04d}'.format(best_epoch), 'Val Loss: {:.6f}'.format(val_loss_values[best_epoch]), 'Val Acc: {:.6f}'.format(val_acc_values[best_epoch]))
    return val_acc_values[best_epoch], val_loss_values[best_epoch]

In [8]:
def select_layers_readout(layers,idx):
    if idx == 1:
        return [i for i in range(layers)]
    elif idx == 2:
        return [0]
    else:
        return [layers-1]

In [9]:
import json

def save_to_json(data):
    json_string = json.dumps(data, indent=4)

    with open('cv_results.json', 'w') as file:
        file.write(json_string)

In [10]:
combination_matrix = []
layers = [2,3]
pool_ratio = [0.2,0.5,0.8]
node_inf_score_metric = ["man","euc"]
layers_readout = [1,2,3]

for l in layers:
    for p in pool_ratio:
        for s_m in node_inf_score_metric:
            for lr in layers_readout:
                lr = select_layers_readout(l,lr)
                args['num_layers'] = l
                args['pooling_ratio'] = p
                args['dist'] = s_m
                args['layers_readout'] = lr
                combination_matrix.append(args.copy())
                

In [11]:
print(len(combination_matrix))
combination_matrix[0]

36


{'device': 'cuda',
 'seed': 777,
 'batch_size': 64,
 'lr': 0.0001,
 'weight_decay': 0.001,
 'epochs': 1000,
 'patience': 100,
 'nhid': 128,
 'pooling_ratio': 0.2,
 'dropout_ratio': 0.5,
 'sample_neighbor': True,
 'sparse_attention': True,
 'structure_learning': True,
 'negative_slope': 0.2,
 'lamb': 1.0,
 'dist': 'man',
 'num_layers': 2,
 'layers_readout': [0, 1],
 'num_classes': 2,
 'num_features': 89}

In [12]:
try:
    with open('cv_results.json', 'r') as file:
        data = json.load(file)
        
except:
    data = []
already_trained = len(data)
print(already_trained)
print(data[-1])
print(type(data[-1]))

14
{'arguments': {'device': 'cuda', 'seed': 777, 'batch_size': 64, 'lr': 0.0001, 'weight_decay': 0.001, 'epochs': 1000, 'patience': 100, 'nhid': 128, 'pooling_ratio': 0.8, 'dropout_ratio': 0.5, 'sample_neighbor': True, 'sparse_attention': True, 'structure_learning': True, 'negative_slope': 0.2, 'lamb': 1.0, 'dist': 'man', 'num_layers': 2, 'layers_readout': [0], 'num_classes': 2, 'num_features': 89}, 'average fold accuracy': 0.7624811763663744, 'average fold loss': 1.9839464962482452}
<class 'dict'>


In [13]:
from google.colab import files
def download_file():
    files.download('cv_results.json')

ModuleNotFoundError: No module named 'google'

In [14]:
for i in range(already_trained,len(combination_matrix)):
    results = []
    args = combination_matrix[i]
    for fold, (train_idx, val_idx) in enumerate(kfold.split(np.arange(len(train_dataset)))):
        val_acc, val_loss = train_fold(fold, train_idx, val_idx)
        results.append((val_acc, val_loss))
    avg_fold_acc,avg_fold_loss = np.array(results).mean(axis=0)
    data.append({'arguments':args, 'average fold accuracy':avg_fold_acc,'average fold loss': avg_fold_loss})
    save_to_json(data)
    print(f"Cross-validation results: {avg_fold_acc} / {avg_fold_loss}, with the args {args}")
    #if 'google.colab' in sys.modules:
        # download_file() I commented this out because it does not seem to work. I will download the file manually

{'device': 'cuda', 'seed': 777, 'batch_size': 64, 'lr': 0.0001, 'weight_decay': 0.001, 'epochs': 1000, 'patience': 100, 'nhid': 128, 'pooling_ratio': 0.8, 'dropout_ratio': 0.5, 'sample_neighbor': True, 'sparse_attention': True, 'structure_learning': True, 'negative_slope': 0.2, 'lamb': 1.0, 'dist': 'man', 'num_layers': 2, 'layers_readout': [1], 'num_classes': 2, 'num_features': 89}


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.21 GiB. GPU 